In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
def TSVtoList(tsvFile):
    import csv
    #Create a container for the tsv
    tsvList = []
    #Open the .tsv file and append it to a python list
    with open(tsvFile,'rb') as tsvIn:
        tsvIn = csv.reader(tsvIn, delimiter='\t')
        for row in tsvIn:
            tsvList.append(row)
    return tsvList

In [3]:
def ExcludeNonImageRows(tsvList):
    index = []
    currentIndex = 1
    mediaNameIndex = GetIndexForHeader("MediaName")
    for row in tsvList[1:]:
        if row[mediaNameIndex] == "":
            index.append(currentIndex)
        currentIndex += 1
    for row in reversed(index):
        del tsvList[row]
    return tsvList

In [77]:
def ExcludeNonAOIRows(tsvList):
    prevFixationHit = "0"
    currentFixationHit = "0"
    saveRow = False
    returnList = []
    for row in tsvList:
        if row[GetIndexForHeader("FixationIndex")] != "":
            prevFixationHit = currentFixationHit
            currentFixationHit = row[GetIndexForHeader("AOI[AOI]Hit")]
        if prevFixationHit == "0" and currentFixationHit == "1":
            saveRow = True
        if prevFixationHit == "1" and currentFixationHit == "0":
            saveRow = False
        if saveRow is True:
            returnList.append(row)
    return returnList

In [55]:
def DivideIntoSeparateImageLists(tsvList):
    imageDictionary = {}
    mediaNameIndex = GetIndexForHeader("MediaName")
    for row in tsvList[1:]:
        if row[113] not in imageDictionary:
            imageDictionary[row[mediaNameIndex]] = [row]
        else:
            imageDictionary[row[mediaNameIndex]].append(row)
    return imageDictionary

In [56]:
def CalculateDurationForImage(dataList):
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    start =  float(dataList[0][RecordingTimestampIndex])
    end = float(dataList[-1][RecordingTimestampIndex])
    duration = end - start
    #Convert to seconds
    duration = duration / 1000
    return duration

In [57]:
def CalculateAmountOfFixations(dataList):
    index = 0
    FixationIndexIndex = GetIndexForHeader("FixationIndex")
    for row in dataList:
        if dataList[index][FixationIndexIndex] is not "":
            break
        index += 1
    start = int(dataList[index][FixationIndexIndex])
    index = -1
    for row in reversed(dataList):
        if dataList[index][FixationIndexIndex] is not "":
            break
        index -= 1
    end = int(dataList[index][FixationIndexIndex])
    amount = end - start
    return amount

In [58]:
def CalculateFixationDuration(dataList):
    index = 0
    FixationIndexIndex = GetIndexForHeader("FixationIndex")
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    prev = ""
    current = ""
    start = 0
    end = 0
    fixationDurations = []
    for row in dataList:
        prev = current
        current = row[FixationIndexIndex]
        if prev is "" and current is not "":
            start = row[RecordingTimestampIndex]
        elif prev is not "" and current is "":
            end = row[RecordingTimestampIndex]
            duration = int(end) - int(start)
            duration = float(duration) / 1000
            fixationDurations.append(duration)
    return fixationDurations

In [59]:
def CalculateAmountOfFixationsPerSecond(duration, amountOfFixations):
    duration = float(duration)
    amountOfFixations = float(amountOfFixations)
    fixationsPerSecond = amountOfFixations/duration
    return fixationsPerSecond

In [60]:
def CalculateAmountOfSaccades(dataList):
    saccadeList = []
    SaccadeIndexIndex = GetIndexForHeader("SaccadeIndex")
    for row in dataList:
        if row[145] is not "":
            saccadeList.append(row[SaccadeIndexIndex])
    start = int(min(saccadeList))
    end = int(max(saccadeList))
    amountOfSaccades = end - start
    return amountOfSaccades

In [61]:
def CalculateDurationOfSaccades(dataList):
    saccadeDurationList = []
    tempSaccadeDurationList = []
    RecordingTimestampIndex = GetIndexForHeader("RecordingTimestamp")
    for row in dataList:
        if row[145] is not "":
            tempSaccadeDurationList.append(int(row[RecordingTimestampIndex]))
        else:
            if len(tempSaccadeDurationList) is not 0:
                saccadeStart = tempSaccadeDurationList[0]
                saccadeEnd = tempSaccadeDurationList[-1]
                saccadeDuration = saccadeEnd - saccadeStart
                saccadeDurationList.append(saccadeDuration)
                tempSaccadeDurationList = []
    if len(tempSaccadeDurationList) is not 0:
        sacadeStart = tempSaccadeDurationList[0]
        saccadeEnd = tempSaccadeDurationList[-1]
        saccadeDuration = saccadeEnd - saccadeStart
        saccadeDurationList.append(saccadeDuration)
        
    return saccadeDurationList

In [62]:
def CalculateVelocityOfSaccades(dataList):
    listOfVelocities = []
    startIndex = 0
    SaccadicAmplitudeIndex = GetIndexForHeader("SaccadicAmplitude")
    if dataList[0][SaccadicAmplitudeIndex] is not "":
        for row in dataList:
            if row[SaccadicAmplitudeIndex] is "":
                break
            else:
                startIndex += 1
    previous = ""
    current = ""
    for row in dataList[startIndex:]:
        previous = current
        current = row[SaccadicAmplitudeIndex]
        if previous is "" and current is not "":
            #convert number because it uses comma and needs to use punctuation
            number = float(row[SaccadicAmplitudeIndex].replace(',','.'))
            listOfVelocities.append(number)
    return listOfVelocities

In [63]:
def CalculatePupilDilation(dataList):
    pupilLeft = []
    pupilRight = []
    PupilLeftIndex = GetIndexForHeader("PupilLeft")
    for row in dataList:
        pupilLeft.append(float(row[PupilLeftIndex].replace(',','.')))
        pupilRight.append(float(row[PupilLeftIndex + 1].replace(',','.')))
    return pupilLeft, pupilRight

In [64]:
def GetIndexForHeader(name):
    indexForHeader = dataHeaders.index(name)
    return indexForHeader

In [65]:
def CalculateAccuracy(dataList):
    print dataList[0]

In [66]:
filePath = "C:/Users/Simon/Documents/Skole/Master/participants/processed/p764b@pupil@study_data_processed.tsv"

dataList = TSVtoList(filePath)

dataHeaders = dataList[0]

dataList = ExcludeNonImageRows(dataList)

dataDictionary = DivideIntoSeparateImageLists(dataList)

print(dataDictionary.keys())

IOError: [Errno 2] No such file or directory: 'C:/Users/Simon/Documents/Skole/Master/participants/processed/p764b@pupil@study_data_processed.tsv'

In [ ]:
index = 0
for header in dataHeaders:
    print str(index) + ": " + header
    index += 1 

In [ ]:
duration = CalculateDurationForImage(dataDictionary["M4A-Q24.png"])
print duration

In [ ]:
amountOfFixations = CalculateAmountOfFixations(dataDictionary["M4A-Q24.png"])
print amountOfFixations

In [ ]:
fixationDuration = CalculateFixationDuration(dataDictionary["M4A-Q24.png"])
print np.mean(fixationDuration)

In [67]:
fixationsPerSecond = CalculateAmountOfFixationsPerSecond(duration, amountOfFixations)
print fixationsPerSecond

3.5267537334


In [68]:
amountOfSaccades = CalculateAmountOfSaccades(dataDictionary["M4A-Q24.png"])
print amountOfSaccades

66


In [69]:
saccadeDurationList = CalculateDurationOfSaccades(dataDictionary["M4A-Q24.png"])
print np.mean(saccadeDurationList)

27.8059701493


In [70]:
saccadeVelocityList = CalculateVelocityOfSaccades(dataDictionary["M4A-Q24.png"])
print np.mean(saccadeVelocityList)

3.9790625


In [71]:
import numpy as np
pupilDilationLeftList, pupilDilationRightList = CalculatePupilDilation(dataDictionary["M4A-Q24.png"])
print np.mean(pupilDilationLeftList), np.var(pupilDilationLeftList), np.std(pupilDilationLeftList)
print np.mean(pupilDilationRightList), np.var(pupilDilationRightList), np.std(pupilDilationRightList)

ValueError: could not convert string to float: 

In [82]:
import os

#Variables to hold the data
data = {}
dataHeaders = []

filePath = "C:/Users/Simon/Documents/Skole/Master/participants/processed/"

filePath = "/Users/SMC/Documents/Skole/MasterThesis/finaltest/"

fileList = os.listdir(filePath)
fileList = fileList[1:]

for participant in fileList:
    #participantshort = participant[:5]
    
    newPath = filePath + participant
    
    dataList = TSVtoList(newPath)

    dataHeaders = dataList[0]
    
    participantshort = dataList[1][GetIndexForHeader("ParticipantName")]
    print participantshort

    dataListExcluded = ExcludeNonImageRows(dataList)
    
    dataListExcluded = ExcludeNonAOIRows(dataListExcluded)
    
    dataList = dataList[1:]

    dataDictionary = DivideIntoSeparateImageLists(dataListExcluded)
    
    aORb = participantshort[-1]
    dataList = dataList[-1]
    
    answerKey = {"M1X-Q01":"True",
                "M1X-Q02":"True",
                "M1X-Q03":"True",
                "M1X-Q04":"False",
                "M1X-Q05":"False",
                "M1X-Q06":"False",
                "M2X-Q07":"False",
                "M2X-Q08":"True",
                "M2X-Q09":"True",
                "M2X-Q10":"False",
                "M2X-Q11":"False",
                "M2X-Q12":"False",
                "M3X-Q13":"True",
                "M3X-Q14":"False",
                "M3X-Q15":"True",
                "M3X-Q16":"False",
                "M3X-Q17":"True",
                "M3X-Q18":"False",
                "M4X-Q19":"True",
                "M4X-Q20":"False",
                "M4X-Q21":"True",
                "M4X-Q22":"False",
                "M4X-Q23":"True",
                "M4X-Q24":"False"}
    
    for image in dataDictionary:
        if image not in data:
            data[image] = {}
            data[image]["duration"] = []
            data[image]["accuracy"] = []
            data[image]["amountOfFixations"] = []
            data[image]["fixationDuration"] = []
            data[image]["fixationsPerSecond"] = []
            data[image]["amountOfSaccades"] = []
            data[image]["saccadeDurationList"] = []
            data[image]["saccadeVelocityList"] = []
            data[image]["pupilDilationLeftList"] = []
            data[image]["pupilDilationRightList"] = []
            data[image]["pupilDilationMean"] = []
        duration = CalculateDurationForImage(dataDictionary[image])
        data[image]["duration"].append(duration)
        amountOfFixations = CalculateAmountOfFixations(dataDictionary[image])
        data[image]["amountOfFixations"].append(amountOfFixations)
        fixationDuration = CalculateFixationDuration(dataDictionary[image])
        data[image]["fixationDuration"].append(np.mean(fixationDuration))
        fixationsPerSecond = CalculateAmountOfFixationsPerSecond(duration, amountOfFixations)
        data[image]["fixationsPerSecond"].append(fixationsPerSecond)
        amountOfSaccades = CalculateAmountOfSaccades(dataDictionary[image])
        data[image]["amountOfSaccades"].append(amountOfSaccades)
        saccadeDurationList = CalculateDurationOfSaccades(dataDictionary[image])
        data[image]["saccadeDurationList"].append(saccadeDurationList)
        saccadeVelocityList = CalculateVelocityOfSaccades(dataDictionary[image])
        data[image]["saccadeVelocityList"].append(np.mean(saccadeVelocityList))
        #pupilDilationLeftList, pupilDilationRightList = CalculatePupilDilation(dataDictionary[image])
        #data[image]["pupilDilationLeftList"].append(np.std(pupilDilationLeftList))
        #data[image]["pupilDilationRightList"].append(np.std(pupilDilationRightList))
        #data[image]["pupilDilationMean"].append(np.mean([np.std(pupilDilationLeftList),np.std(pupilDilationRightList)]))
        
        imageShort = image[:-4]
        if imageShort[0] != "T":
            answer = dataList[GetIndexForHeader("["+imageShort+"]Value")]
            if answer[0] == " ":
                answer = answer[1:]
            pre = imageShort[:2]
            post = imageShort[-4:]
            if answerKey[pre+"X"+post] == answer:
                data[image]["accuracy"].append(1)
            else:
                data[image]["accuracy"].append(0)


p665a


In [20]:
for row in dataHeaders:
    print row

﻿ExportDate
StudioVersionRec
StudioProjectName
StudioTestName
ParticipantName
[M1BE-Q1]Value
[M1BE-Q2]Value
[M1BE-Q3]Value
[M1BE-Q4]Value
[M1BE-Q5]Value
[M1BE-Q6]Value
[M1B-Q01]Value
[M1B-Q02]Value
[M1B-Q03]Value
[M1B-Q04]Value
[M1B-Q05]Value
[M1B-Q06]Value
[M2BE-Q10]Value
[M2BE-Q11]Value
[M2BE-Q12]Value
[M2BE-Q7]Value
[M2BE-Q8]Value
[M2BE-Q9]Value
[M2B-Q07]Value
[M2B-Q08]Value
[M2B-Q09]Value
[M2B-Q10]Value
[M2B-Q11]Value
[M2B-Q12]Value
[M3BE-Q13]Value
[M3BE-Q14]Value
[M3BE-Q15]Value
[M3BE-Q16]Value
[M3BE-Q17]Value
[M3BE-Q18]Value
[M3B-Q13]Value
[M3B-Q14]Value
[M3B-Q15]Value
[M3B-Q16]Value
[M3B-Q17]Value
[M3B-Q18]Value
[M4BE-Q19]Value
[M4BE-Q20]Value
[M4BE-Q21]Value
[M4BE-Q22]Value
[M4BE-Q23]Value
[M4BE-Q24]Value
[M4B-Q19]Value
[M4B-Q20]Value
[M4B-Q21]Value
[M4B-Q22]Value
[M4B-Q23]Value
[M4B-Q24]Value
[TUTB-Q01]Value
[TUTB-Q02]Value
[TUTB-Q03]Value
[TUTA-Q01]Value
[TUTA-Q02]Value
[TUTA-Q03]Value
[M1A-Q01]Value
[M1AE-Q1]Value
[M1A-Q02]Value
[M1AE-Q2]Value
[M1A-Q03]Value
[M1AE-Q3]Value
[

In [29]:
for row in dataDictionary["M1A-Q01.png"]:
    print row[GetIndexForHeader("FixationIndex")], row[GetIndexForHeader("SaccadeIndex")], row[GetIndexForHeader("AOI[AOI]Hit")]

211  
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
211  0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
 224 0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
212  0
 225 0
 225 0
 225 0
 225 0
 225 0
 225 0
 225 0
213  0
213  0
213  0
213  0
213  0
213  0
213  0


In [ ]:
measure = "pupilDilationMean"
TUT = data["TUTX-Q01.png"][measure] + data["TUTX-Q02.png"][measure] + data["TUTX-Q03.png"][measure]
tutMean = np.mean(TUT)

for image in data.keys():
    data[image]["pupilDilationDiff"] = []
    for index in xrange(len(data[image]["pupilDilationMean"])):
        data[image]["pupilDilationDiff"].append(float(data[image]["pupilDilationMean"][index])-float(TUT[index]))

In [ ]:
from pylab import *
import matplotlib.pyplot as plt
print data['M4A-Q24.png'].keys()

In [ ]:
def getModelData(model, measure, localAndGlobal="Both"):
    multiplier = int(model[1])
    aORb = model[2]
    end = 6 * multiplier
    start = end - 5
    if localAndGlobal is "Local":
        end = end - 3
    elif localAndGlobal is "Global":
        start = start + 3
    prefix = model +"-"
    questions = []
    for index in range(start, end+1):
        question = index
        question = "%02d" % (question,)
        question = "Q"+question
        questions.append(question)
    
    collecter = []
    for question in questions:
        collecter += data[prefix+question+".png"][measure]
    
    return collecter

print getModelData("M1A", "duration")
%matplotlib inline

In [ ]:
%matplotlib inline
measure = "duration"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]





def boxplot(figureSubtitle, ylabel, MA, MB):
    fig = plt.figure(figsize=(9,6))
    #fig.suptitle(figureSubtitle, fontsize=14, fontweight='bold')

    ax1A = fig.add_subplot(121)
    ax1A.set_ylabel(ylabel)
    ax1A.set_xlabel("Group A")
    ax1A.tick_params(axis="x", which="both", bottom="off", top="off", labelbottom="off")
    bp1A = ax1A.boxplot(MA)

    ax1B = fig.add_subplot(122, sharey = ax1A)
    ax1B.set_ylabel(ylabel)
    ax1B.set_xlabel("Group B")
    ax1B.tick_params(axis="x", which="both", bottom="off", top="off", labelbottom="off")
    bp1B = ax1B.boxplot(MB)
    
    return fig
db = "DurationBoxplot"    
m1 = boxplot("Model 1 duration", "Time (sec)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 duration", "Time (sec)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 duration", "Time (sec)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 duration", "Time (sec)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "amountOfFixations"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "amountOfFixationsBoxplot"    
m1 = boxplot("Model 1 fixation amount", "Fixations", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 fixation amount", "Fixations", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 fixation amount", "Fixations", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 fixation amount", "Fixations", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "fixationsPerSecond"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "fixationsPerSecondBoxplot"    
m1 = boxplot("Model 1 fixations per second", "Fixations (sec)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 fixations per second", "Fixations (sec)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 fixations per second", "Fixations (sec)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 fixations per second", "Fixations (sec)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "amountOfSaccades"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "amountOfSaccadesBoxplot"    
m1 = boxplot("Model 1 amount of saccades", "Saccades", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 amount of saccades", "Saccades", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 amount of saccades", "Saccades", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 amount of saccades", "Saccades", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "fixationDuration"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "fixationDurationBoxplot"    
m1 = boxplot("Model 1 fixation duration", "Duration (sec)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 fixation duration", "Duration (sec)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 fixation duration", "Duration (sec)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 fixation duration", "Duration (sec)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "saccadeVelocityList"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "saccadeVelocityBoxplot"    
m1 = boxplot("Model 1 saccade velocity", "Velocity", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 saccade velocity", "Velocity", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 saccade velocity", "Velocity", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 saccade velocity", "Velocity", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "pupilDilationLeftList"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "pupilDilationLeftListBoxplot"    
m1 = boxplot("Model 1 pupil dilation (left eye)", "Dilation (std)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 pupil dilation (left eye)", "Dilation (std)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 pupil dilation (left eye)", "Dilation (std)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 pupil dilation (left eye)", "Dilation (std)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "pupilDilationRightList"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "pupilDilationRightListBoxplot"    
m1 = boxplot("Model 1 pupil dilation (right eye)", "Dilation (std)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 pupil dilation (right eye)", "Dilation (std)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 pupil dilation (right eye)", "Dilation (std)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 pupil dilation (right eye)", "Dilation (std)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "pupilDilationMean"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "pupilDilationMeanBoxplot"    
m1 = boxplot("Model 1 pupil dilation", "Dilation (std)", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 pupil dilation", "Dilation (std)", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 pupil dilation", "Dilation (std)", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 pupil dilation", "Dilation (std)", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "pupilDilationDiff"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]


db = "pupilDilationDiffBoxplot"    
m1 = boxplot("Model 1 pupil dilation", "Dilation difference", M1A, M1B)
m1.savefig("M1"+db)
m2 = boxplot("Model 2 pupil dilation", "Dilation difference", M2A, M2B)
m2.savefig("M2"+db)
m3 = boxplot("Model 3 pupil dilation", "Dilation difference", M3A, M3B)
m3.savefig("M3"+db)
m4 = boxplot("Model 4 pupil dilation", "Dilation difference", M4A, M4B)
m4.savefig("M4"+db)

In [ ]:
measure = "fixationsPerSecond"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A

print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))

In [ ]:
measure = "fixationDuration"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A

print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))

In [ ]:
measure = "saccadeVelocityList"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A
print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))

In [ ]:
measure = "amountOfFixations"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A
print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))

In [ ]:
measure = "pupilDilationMean"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

TUT = data["TUTX-Q01.png"][measure] + data["TUTX-Q02.png"][measure] + data["TUTX-Q03.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A
print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))
print ""
print "TUT \t" + str(np.mean(TUT)) + "\t" + str(np.var(TUT)) + "\t" + str(np.std(TUT)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))


In [ ]:
measure = "pupilDilationDiff"
M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

messyDirection = M1B + M3A
nonMessyDirection = M1A + M3B
messyVisibility = M2A + M4B
nonMessyVisibility = M2B + M4A
print len(messyDirection)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy direction \t|" + str(np.mean(messyDirection)) + "\t|" + str(np.var(messyDirection)) + "\t|" + str(np.std(messyDirection)) + "\t" + str(min(messyDirection)) + "\t" + str(max(messyDirection))
print "Non messy direction \t|" + str(np.mean(nonMessyDirection)) + "\t|" + str(np.var(nonMessyDirection)) + "\t|" + str(np.std(nonMessyDirection)) + "\t" + str(min(nonMessyDirection)) + "\t" + str(max(nonMessyDirection))
print "Messy visibility \t|" + str(np.mean(messyVisibility)) + "\t|" + str(np.var(messyVisibility)) + "\t|" + str(np.std(messyVisibility)) + "\t" + str(min(messyVisibility)) + "\t" + str(max(messyVisibility))
print "Non messy visibility \t|" + str(np.mean(nonMessyVisibility)) + "\t|" + str(np.var(nonMessyVisibility)) + "\t|" + str(np.std(nonMessyVisibility)) + "\t" + str(min(nonMessyVisibility)) + "\t" + str(max(nonMessyVisibility))

In [ ]:
measure = "fixationsPerSecond"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL + messyL
nonMessyG = M1AG + M3BG + M2BG + M4AG + messyG

print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG)) + "\t" + str(min(nonMessyG)) + "\t" + str(max(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL)) + "\t" + str(min(nonMessyL)) + "\t" + str(max(nonMessyL))

In [ ]:
measure = "fixationDuration"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL + messyL
nonMessyG = M1AG + M3BG + M2BG + M4AG + messyG

print measure + "\t|Mean\t\t|Variance\t\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG)) + "\t" + str(min(nonMessyG)) + "\t" + str(max(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL)) + "\t" + str(min(nonMessyL)) + "\t" + str(max(nonMessyL))

In [ ]:
measure = "saccadeVelocityList"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL + messyL
nonMessyG = M1AG + M3BG + M2BG + M4AG + messyG

print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG)) + "\t" + str(min(nonMessyG)) + "\t" + str(max(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL)) + "\t" + str(min(nonMessyL)) + "\t" + str(max(nonMessyL))

In [ ]:
measure = "duration"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL
nonMessyG = M1AG + M3BG + M2BG + M4AG

print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL))

In [ ]:
measure = "amountOfFixations"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL + messyL
nonMessyG = M1AG + M3BG + M2BG + M4AG + messyG
print len(nonMessyG)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG)) + "\t" + str(min(nonMessyG)) + "\t" + str(max(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL)) + "\t" + str(min(nonMessyL)) + "\t" + str(max(nonMessyL))

In [ ]:
measure = "pupilDilationDiff"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL + messyL
nonMessyG = M1AG + M3BG + M2BG + M4AG + messyG
print len(nonMessyG)
print measure + "\t|Mean\t\t|Variance\t|Standard diviation"
print "Messy Global \t\t|" + str(np.mean(messyG)) + "\t|" + str(np.var(messyG)) + "\t|" + str(np.std(messyG))
print "Messy Local \t\t|" + str(np.mean(messyL)) + "\t|" + str(np.var(messyL)) + "\t|" + str(np.std(messyL))
print "Non Messy Global \t|" + str(np.mean(nonMessyG)) + "\t|" + str(np.var(nonMessyG)) + "\t|" + str(np.std(nonMessyG)) + "\t" + str(min(nonMessyG)) + "\t" + str(max(nonMessyG))
print "Non Messy Local \t|" + str(np.mean(nonMessyL)) + "\t|" + str(np.var(nonMessyL)) + "\t|" + str(np.std(nonMessyL)) + "\t" + str(min(nonMessyL)) + "\t" + str(max(nonMessyL))

In [ ]:
measure = "fixationsPerSecond"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL
print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measure = "fixationDuration"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL
print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measure = "saccadeVelocityList"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measure = "amountOfFixations"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL
print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
print data["M3B-Q15.png"]["duration"]

In [ ]:
measure = "duration"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measure = "accuracy"
M1AL = []
for index in xrange(len(data["M1A-Q01.png"][measure])):
    M1AL.append(data["M1A-Q01.png"][measure][index]+data["M1A-Q02.png"][measure][index]+data["M1A-Q03.png"][measure][index])
M1AG = []
for index in xrange(len(data["M1A-Q04.png"][measure])):
    M1AG.append(data["M1A-Q04.png"][measure][index]+data["M1A-Q05.png"][measure][index]+data["M1A-Q06.png"][measure][index])
M1BL = []
for index in xrange(len(data["M1B-Q01.png"][measure])):
    M1BL.append(data["M1B-Q01.png"][measure][index]+data["M1B-Q02.png"][measure][index]+data["M1B-Q03.png"][measure][index])
M1BG = []
for index in xrange(len(data["M1B-Q04.png"][measure])):
    M1BG.append(data["M1B-Q04.png"][measure][index]+data["M1B-Q05.png"][measure][index]+data["M1B-Q06.png"][measure][index])
M2AL = []
for index in xrange(len(data["M2A-Q07.png"][measure])):
    M2AL.append(data["M2A-Q07.png"][measure][index]+data["M2A-Q08.png"][measure][index]+data["M2A-Q09.png"][measure][index])
M2AG = []
for index in xrange(len(data["M2A-Q10.png"][measure])):
    M2AG.append(data["M2A-Q10.png"][measure][index]+data["M2A-Q11.png"][measure][index]+data["M2A-Q12.png"][measure][index])
M2BL = []
for index in xrange(len(data["M2B-Q07.png"][measure])):
    M2BL.append(data["M2B-Q07.png"][measure][index]+data["M2B-Q08.png"][measure][index]+data["M2B-Q09.png"][measure][index])
M2BG = []
for index in xrange(len(data["M2B-Q10.png"][measure])):
    M2BG.append(data["M2B-Q10.png"][measure][index]+data["M2B-Q11.png"][measure][index]+data["M2B-Q12.png"][measure][index])
M3AL = []
for index in xrange(len(data["M3A-Q13.png"][measure])):
    M3AL.append(data["M3A-Q13.png"][measure][index]+data["M3A-Q14.png"][measure][index]+data["M3A-Q15.png"][measure][index])
M3AG = []
for index in xrange(len(data["M3A-Q16.png"][measure])):
    M3AG.append(data["M3A-Q16.png"][measure][index]+data["M3A-Q17.png"][measure][index]+data["M3A-Q18.png"][measure][index])
M3BL = []
for index in xrange(len(data["M3B-Q13.png"][measure])):
    M3BL.append(data["M3B-Q13.png"][measure][index]+data["M3B-Q14.png"][measure][index]+data["M3B-Q15.png"][measure][index])
M3BG = []
for index in xrange(len(data["M3B-Q16.png"][measure])):
    M3BG.append(data["M3B-Q16.png"][measure][index]+data["M3B-Q17.png"][measure][index]+data["M3B-Q18.png"][measure][index])
M4AL = []
for index in xrange(len(data["M4A-Q19.png"][measure])):
    M4AL.append(data["M4A-Q19.png"][measure][index]+data["M4A-Q20.png"][measure][index]+data["M4A-Q24.png"][measure][index])
M4AG = []
for index in xrange(len(data["M4A-Q22.png"][measure])):
    M4AG.append(data["M4A-Q22.png"][measure][index]+data["M4A-Q23.png"][measure][index]+data["M4A-Q21.png"][measure][index])
M4BL = []
for index in xrange(len(data["M4B-Q19.png"][measure])):
    M4BL.append(data["M4B-Q19.png"][measure][index]+data["M4B-Q20.png"][measure][index]+data["M4B-Q24.png"][measure][index])
M4BG = []
for index in xrange(len(data["M4B-Q22.png"][measure])):
    M4BG.append(data["M4B-Q22.png"][measure][index]+data["M4B-Q23.png"][measure][index]+data["M4B-Q21.png"][measure][index])

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL
print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measure = "pupilDilationDiff"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL
print len(messyDirectionGlobal)
print measure + "\t|Mean\t\t|Variance\t|Standard deviation"
print "Messy direction global \t|" + str(np.mean(messyDirectionGlobal)) + "\t|" + str(np.var(messyDirectionGlobal)) + "\t|" + str(np.std(messyDirectionGlobal)) + "\t" + str(min(messyDirectionGlobal)) + "\t" + str(max(messyDirectionGlobal))
print "Non messy direction global \t|" + str(np.mean(nonMessyDirectionGlobal)) + "\t|" + str(np.var(nonMessyDirectionGlobal)) + "\t|" + str(np.std(nonMessyDirectionGlobal)) + "\t" + str(min(nonMessyDirectionGlobal)) + "\t" + str(max(nonMessyDirectionGlobal))
print "Messy direction local \t|" + str(np.mean(messyDirectionLocal)) + "\t|" + str(np.var(messyDirectionLocal)) + "\t|" + str(np.std(messyDirectionLocal)) + "\t" + str(min(messyDirectionLocal)) + "\t" + str(max(messyDirectionLocal))
print "Non messy direction local \t|" + str(np.mean(nonMessyDirectionLocal)) + "\t|" + str(np.var(nonMessyDirectionLocal)) + "\t|" + str(np.std(nonMessyDirectionLocal)) + "\t" + str(min(nonMessyDirectionLocal)) + "\t" + str(max(nonMessyDirectionLocal))
print "Messy visibility global \t|" + str(np.mean(messyVisibilityGlobal)) + "\t|" + str(np.var(messyVisibilityGlobal)) + "\t|" + str(np.std(messyVisibilityGlobal)) + "\t" + str(min(messyVisibilityGlobal)) + "\t" + str(max(messyVisibilityGlobal))
print "Non messy visibility global \t|" + str(np.mean(nonMessyVisibilityGlobal)) + "\t|" + str(np.var(nonMessyVisibilityGlobal)) + "\t|" + str(np.std(nonMessyVisibilityGlobal)) + "\t" + str(min(nonMessyVisibilityGlobal)) + "\t" + str(max(nonMessyVisibilityGlobal))
print "Messy visibility local \t|" + str(np.mean(messyVisibilityLocal)) + "\t|" + str(np.var(messyVisibilityLocal)) + "\t|" + str(np.std(messyVisibilityLocal)) + "\t" + str(min(messyVisibilityLocal)) + "\t" + str(max(messyVisibilityLocal))
print "Non messy visibility local \t|" + str(np.mean(nonMessyVisibilityLocal)) + "\t|" + str(np.var(nonMessyVisibilityLocal)) + "\t|" + str(np.std(nonMessyVisibilityLocal)) + "\t" + str(min(nonMessyVisibilityLocal)) + "\t" + str(max(nonMessyVisibilityLocal))

In [ ]:
measureList = ["amountOfFixations", "fixationsPerSecond", "fixationDuration", "saccadeVelocityList", "pupilDilationDiff"]

In [ ]:
#high change of direction
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
    M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

    collectedLists = M1B+M3A
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
#low change of direction
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
    M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

    collectedLists = M1A + M3B
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
#low block visibility
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
    M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

    collectedLists = M2A + M4B
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
#high block visibility
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1A = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] +data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1B = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] +data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2A = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] +data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2B = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] +data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3A = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] +data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3B = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] +data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4A = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q21.png"][measure] +data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q24.png"][measure]
    M4B = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q21.png"][measure] +data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q24.png"][measure]

    collectedLists = M2B + M4A
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
#local questions
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyL = M1BL + M3AL + M2AL + M4BL
    messyG = M1BG + M3AG + M2AG + M4BG
    nonMessyL = M1AL + M3BL + M2BL + M4AL
    nonMessyG = M1AG + M3BG + M2BG + M4AG
    
    collectedLists = messyL + nonMessyL
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
#global questions
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyL = M1BL + M3AL + M2AL + M4BL
    messyG = M1BG + M3AG + M2AG + M4BG
    nonMessyL = M1AL + M3BL + M2BL + M4AL
    nonMessyG = M1AG + M3BG + M2BG + M4AG
    
    collectedLists = messyG + nonMessyG
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# high change of direction local
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M1BL + M3AL
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# high change of direction global
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M1BG + M3AG
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# low change of direction local
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M1AL + M3BL
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# low change of direction gobal
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M1AG + M3BG
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# low block visibility local
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M2AL + M4BL
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# low block visibility global
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M2AG + M4BG
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# high block visibility local
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M2BL + M4AL
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
# high block visibility global
#measure = "fixationsPerSecond"
#fixationsPerSecond = M1B + M3A

for measurement in measureList:
    measure = measurement
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL
    
    collectedLists = M2BG + M4AG
    statistic, pval = stats.shapiro(collectedLists)
    verdict = ""
    if(pval < 0.055):
        verdict = "No"
    else:
        verdict = "Yes"
    
    print measure + "\t" + "%.3f" % (statistic,) + "\t" + str(len(collectedLists)) + "\t" + "%0.3f" % (pval,) + "\t" + verdict

In [ ]:
measure = "amountOfFixations"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyL = M1BL + M3AL + M2AL + M4BL
messyG = M1BG + M3AG + M2AG + M4BG
nonMessyL = M1AL + M3BL + M2BL + M4AL
nonMessyG = M1AG + M3BG + M2BG + M4AG
allFixations = messyL+messyG+nonMessyL+nonMessyG

stats.shapiro(allFixations)

In [ ]:
measure = "duration"
M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG 
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

print stats.shapiro(messyDirectionGlobal)
print stats.shapiro(messyDirectionLocal)
print stats.shapiro(nonMessyDirectionGlobal)
print stats.shapiro(nonMessyDirectionLocal)
print stats.shapiro(messyVisibilityGlobal)
print stats.shapiro(messyVisibilityLocal)
print stats.shapiro(nonMessyVisibilityGlobal)
print stats.shapiro(nonMessyVisibilityLocal)

In [ ]:
measure = "accuracy"
M1AL = []
for index in xrange(len(data["M1A-Q01.png"][measure])):
    M1AL.append(data["M1A-Q01.png"][measure][index]+data["M1A-Q02.png"][measure][index]+data["M1A-Q03.png"][measure][index])
M1AG = []
for index in xrange(len(data["M1A-Q04.png"][measure])):
    M1AG.append(data["M1A-Q04.png"][measure][index]+data["M1A-Q05.png"][measure][index]+data["M1A-Q06.png"][measure][index])
M1BL = []
for index in xrange(len(data["M1B-Q01.png"][measure])):
    M1BL.append(data["M1B-Q01.png"][measure][index]+data["M1B-Q02.png"][measure][index]+data["M1B-Q03.png"][measure][index])
M1BG = []
for index in xrange(len(data["M1B-Q04.png"][measure])):
    M1BG.append(data["M1B-Q04.png"][measure][index]+data["M1B-Q05.png"][measure][index]+data["M1B-Q06.png"][measure][index])
M2AL = []
for index in xrange(len(data["M2A-Q07.png"][measure])):
    M2AL.append(data["M2A-Q07.png"][measure][index]+data["M2A-Q08.png"][measure][index]+data["M2A-Q09.png"][measure][index])
M2AG = []
for index in xrange(len(data["M2A-Q10.png"][measure])):
    M2AG.append(data["M2A-Q10.png"][measure][index]+data["M2A-Q11.png"][measure][index]+data["M2A-Q12.png"][measure][index])
M2BL = []
for index in xrange(len(data["M2B-Q07.png"][measure])):
    M2BL.append(data["M2B-Q07.png"][measure][index]+data["M2B-Q08.png"][measure][index]+data["M2B-Q09.png"][measure][index])
M2BG = []
for index in xrange(len(data["M2B-Q10.png"][measure])):
    M2BG.append(data["M2B-Q10.png"][measure][index]+data["M2B-Q11.png"][measure][index]+data["M2B-Q12.png"][measure][index])
M3AL = []
for index in xrange(len(data["M3A-Q13.png"][measure])):
    M3AL.append(data["M3A-Q13.png"][measure][index]+data["M3A-Q14.png"][measure][index]+data["M3A-Q15.png"][measure][index])
M3AG = []
for index in xrange(len(data["M3A-Q16.png"][measure])):
    M3AG.append(data["M3A-Q16.png"][measure][index]+data["M3A-Q17.png"][measure][index]+data["M3A-Q18.png"][measure][index])
M3BL = []
for index in xrange(len(data["M3B-Q13.png"][measure])):
    M3BL.append(data["M3B-Q13.png"][measure][index]+data["M3B-Q14.png"][measure][index]+data["M3B-Q15.png"][measure][index])
M3BG = []
for index in xrange(len(data["M3B-Q16.png"][measure])):
    M3BG.append(data["M3B-Q16.png"][measure][index]+data["M3B-Q17.png"][measure][index]+data["M3B-Q18.png"][measure][index])
M4AL = []
for index in xrange(len(data["M4A-Q19.png"][measure])):
    M4AL.append(data["M4A-Q19.png"][measure][index]+data["M4A-Q20.png"][measure][index]+data["M4A-Q24.png"][measure][index])
M4AG = []
for index in xrange(len(data["M4A-Q22.png"][measure])):
    M4AG.append(data["M4A-Q22.png"][measure][index]+data["M4A-Q23.png"][measure][index]+data["M4A-Q21.png"][measure][index])
M4BL = []
for index in xrange(len(data["M4B-Q19.png"][measure])):
    M4BL.append(data["M4B-Q19.png"][measure][index]+data["M4B-Q20.png"][measure][index]+data["M4B-Q24.png"][measure][index])
M4BG = []
for index in xrange(len(data["M4B-Q22.png"][measure])):
    M4BG.append(data["M4B-Q22.png"][measure][index]+data["M4B-Q23.png"][measure][index]+data["M4B-Q21.png"][measure][index])

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

print stats.shapiro(messyDirectionGlobal)
print stats.shapiro(messyDirectionLocal)
print stats.shapiro(nonMessyDirectionGlobal)
print stats.shapiro(nonMessyDirectionLocal)
print stats.shapiro(messyVisibilityGlobal)
print stats.shapiro(messyVisibilityLocal)
print stats.shapiro(nonMessyVisibilityGlobal)
print stats.shapiro(nonMessyVisibilityLocal)

In [ ]:
data["M1A-Q01.png"]["pupilDilationDiff"]

In [ ]:
import scipy.stats as stats
stats.shapiro(M1BG+M3AG+M1AG+M3BG)

In [ ]:
data["M1A-Q01.png"].keys()

In [ ]:
data.keys()

In [ ]:
dataHeaders

In [ ]:
print tutMean

In [ ]:
data["M1"]

In [ ]:
import statsmodels


In [ ]:
def changeMeasure(measure):
    M1AL = data["M1A-Q01.png"][measure] + data["M1A-Q02.png"][measure] + data["M1A-Q03.png"][measure] 
    M1AG = data["M1A-Q04.png"][measure] +data["M1A-Q05.png"][measure] +data["M1A-Q06.png"][measure]
    M1BL = data["M1B-Q01.png"][measure] + data["M1B-Q02.png"][measure] + data["M1B-Q03.png"][measure] 
    M1BG = data["M1B-Q04.png"][measure] +data["M1B-Q05.png"][measure] +data["M1B-Q06.png"][measure]
    M2AL = data["M2A-Q07.png"][measure] + data["M2A-Q08.png"][measure] + data["M2A-Q09.png"][measure] 
    M2AG = data["M2A-Q10.png"][measure] +data["M2A-Q11.png"][measure] +data["M2A-Q12.png"][measure]
    M2BL = data["M2B-Q07.png"][measure] + data["M2B-Q08.png"][measure] + data["M2B-Q09.png"][measure] 
    M2BG = data["M2B-Q10.png"][measure] +data["M2B-Q11.png"][measure] +data["M2B-Q12.png"][measure]
    M3AL = data["M3A-Q13.png"][measure] + data["M3A-Q14.png"][measure] + data["M3A-Q15.png"][measure] 
    M3AG = data["M3A-Q16.png"][measure] +data["M3A-Q17.png"][measure] +data["M3A-Q18.png"][measure]
    M3BL = data["M3B-Q13.png"][measure] + data["M3B-Q14.png"][measure] + data["M3B-Q15.png"][measure] 
    M3BG = data["M3B-Q16.png"][measure] +data["M3B-Q17.png"][measure] +data["M3B-Q18.png"][measure]
    M4AL = data["M4A-Q19.png"][measure] + data["M4A-Q20.png"][measure] + data["M4A-Q24.png"][measure] 
    M4AG = data["M4A-Q22.png"][measure] +data["M4A-Q23.png"][measure] +data["M4A-Q21.png"][measure]
    M4BL = data["M4B-Q19.png"][measure] + data["M4B-Q20.png"][measure] + data["M4B-Q24.png"][measure] 
    M4BG = data["M4B-Q22.png"][measure] +data["M4B-Q23.png"][measure] +data["M4B-Q21.png"][measure]

    global messyDirectionGlobal
    global nonMessyDirectionGlobal
    global messyDirectionLocal
    global nonMessyDirectionLocal
    global messyVisibilityGlobal
    global nonMessyVisibilityGlobal
    global messyVisibilityLocal
    global nonMessyVisibilityLocal
    
    messyDirectionGlobal = M1BG + M3AG
    nonMessyDirectionGlobal = M1AG + M3BG
    messyDirectionLocal = M1BL + M3AL
    nonMessyDirectionLocal = M1AL + M3BL
    messyVisibilityGlobal = M2AG + M4BG
    nonMessyVisibilityGlobal = M2BG + M4AG
    messyVisibilityLocal = M2AL + M4BL
    nonMessyVisibilityLocal = M2BL + M4AL

In [ ]:
def statisticalTest(data1, data2, normalDistributed):       
    statistic = 0
    pval = 0
    if normalDistributed is True:
        #Parametric test
        #statistic, pval = statsmodels.test(data1,data2,paired=True)
        statistic, pval = stats.stats.ttest_rel(data1, data2)
    else: 
        #Non-parametric test
        statistic, pval = stats.wilcoxon(data1,data2)
    return statistic, pval

In [ ]:
#Change of direction - High Global vs Low Global
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Change of direction - High Local vs Low Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Block visibility - High Global vs Low Global
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Change of direction - High Local vs Low Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Type of question - Change of Direction High Global vs Change of Direction High Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Type of question - Change of Direction Low Global vs Change of Direction Low Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Type of question - Block Visibility Low Global vs Block Visibility Low Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
#Type of question - Block visibility High Global vs Block visibility High Local
    #Fixation count
measure = "amountOfFixations"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixations per second
measure = "fixationsPerSecond"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Fixation duration
measure = "fixationDuration"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Saccade velocity
measure = "saccadeVelocityList"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
    #Pupil dilation difference
measure = "pupilDilationDiff"
changeMeasure(measure)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
measure = "duration"
changeMeasure(measure)
statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
measure = "accuracy"
M1AL = []
for index in xrange(len(data["M1A-Q01.png"][measure])):
    M1AL.append(data["M1A-Q01.png"][measure][index]+data["M1A-Q02.png"][measure][index]+data["M1A-Q03.png"][measure][index])
M1AG = []
for index in xrange(len(data["M1A-Q04.png"][measure])):
    M1AG.append(data["M1A-Q04.png"][measure][index]+data["M1A-Q05.png"][measure][index]+data["M1A-Q06.png"][measure][index])
M1BL = []
for index in xrange(len(data["M1B-Q01.png"][measure])):
    M1BL.append(data["M1B-Q01.png"][measure][index]+data["M1B-Q02.png"][measure][index]+data["M1B-Q03.png"][measure][index])
M1BG = []
for index in xrange(len(data["M1B-Q04.png"][measure])):
    M1BG.append(data["M1B-Q04.png"][measure][index]+data["M1B-Q05.png"][measure][index]+data["M1B-Q06.png"][measure][index])
M2AL = []
for index in xrange(len(data["M2A-Q07.png"][measure])):
    M2AL.append(data["M2A-Q07.png"][measure][index]+data["M2A-Q08.png"][measure][index]+data["M2A-Q09.png"][measure][index])
M2AG = []
for index in xrange(len(data["M2A-Q10.png"][measure])):
    M2AG.append(data["M2A-Q10.png"][measure][index]+data["M2A-Q11.png"][measure][index]+data["M2A-Q12.png"][measure][index])
M2BL = []
for index in xrange(len(data["M2B-Q07.png"][measure])):
    M2BL.append(data["M2B-Q07.png"][measure][index]+data["M2B-Q08.png"][measure][index]+data["M2B-Q09.png"][measure][index])
M2BG = []
for index in xrange(len(data["M2B-Q10.png"][measure])):
    M2BG.append(data["M2B-Q10.png"][measure][index]+data["M2B-Q11.png"][measure][index]+data["M2B-Q12.png"][measure][index])
M3AL = []
for index in xrange(len(data["M3A-Q13.png"][measure])):
    M3AL.append(data["M3A-Q13.png"][measure][index]+data["M3A-Q14.png"][measure][index]+data["M3A-Q15.png"][measure][index])
M3AG = []
for index in xrange(len(data["M3A-Q16.png"][measure])):
    M3AG.append(data["M3A-Q16.png"][measure][index]+data["M3A-Q17.png"][measure][index]+data["M3A-Q18.png"][measure][index])
M3BL = []
for index in xrange(len(data["M3B-Q13.png"][measure])):
    M3BL.append(data["M3B-Q13.png"][measure][index]+data["M3B-Q14.png"][measure][index]+data["M3B-Q15.png"][measure][index])
M3BG = []
for index in xrange(len(data["M3B-Q16.png"][measure])):
    M3BG.append(data["M3B-Q16.png"][measure][index]+data["M3B-Q17.png"][measure][index]+data["M3B-Q18.png"][measure][index])
M4AL = []
for index in xrange(len(data["M4A-Q19.png"][measure])):
    M4AL.append(data["M4A-Q19.png"][measure][index]+data["M4A-Q20.png"][measure][index]+data["M4A-Q24.png"][measure][index])
M4AG = []
for index in xrange(len(data["M4A-Q22.png"][measure])):
    M4AG.append(data["M4A-Q22.png"][measure][index]+data["M4A-Q23.png"][measure][index]+data["M4A-Q21.png"][measure][index])
M4BL = []
for index in xrange(len(data["M4B-Q19.png"][measure])):
    M4BL.append(data["M4B-Q19.png"][measure][index]+data["M4B-Q20.png"][measure][index]+data["M4B-Q24.png"][measure][index])
M4BG = []
for index in xrange(len(data["M4B-Q22.png"][measure])):
    M4BG.append(data["M4B-Q22.png"][measure][index]+data["M4B-Q23.png"][measure][index]+data["M4B-Q21.png"][measure][index])

messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

statistic, pval = statisticalTest(messyDirectionGlobal, nonMessyDirectionGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyDirectionLocal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityGlobal, nonMessyVisibilityGlobal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityLocal, nonMessyVisibilityLocal, True)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyDirectionGlobal, messyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(nonMessyDirectionGlobal, nonMessyDirectionLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(messyVisibilityGlobal, messyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)
statistic, pval = statisticalTest(nonMessyVisibilityGlobal, nonMessyVisibilityLocal, False)
print measure, "%.3f" % (statistic,), "%.3f" % (pval,)

In [ ]:
messyDirectionGlobal = M1BG + M3AG
nonMessyDirectionGlobal = M1AG + M3BG
messyDirectionLocal = M1BL + M3AL
nonMessyDirectionLocal = M1AL + M3BL
messyVisibilityGlobal = M2AG + M4BG
nonMessyVisibilityGlobal = M2BG + M4AG
messyVisibilityLocal = M2AL + M4BL
nonMessyVisibilityLocal = M2BL + M4AL

In [ ]:
aResults = [0,0,0,0,0,0,0]
for image in data.keys():
    if image[0] != "T" and image[2] == "A":
        for index in xrange(len(data[image]["accuracy"])):
            aResults[index] += data[image]["accuracy"][index]

In [ ]:
bResults = [0,0,0,0,0,0]
for image in data.keys():
    if image[0] != "T" and image[2] == "B":
        for index in xrange(len(data[image]["accuracy"])):
            bResults[index] += data[image]["accuracy"][index]

In [ ]:

print aResults
print bResults
print np.mean(aResults+bResults)
print np.median(aResults+bResults)